In [ ]:
!pip install esm
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from Bio import SeqIO
from esm import pretrained
from Bio.SeqUtils.ProtParam import ProteinAnalysis

In [ ]:
from datasets import load_dataset

# Load the dataset in streaming mode
streamed_dataset = load_dataset("bloyal/uniref50", split="train", streaming=True)

# Take only the first 25,000 items
from itertools import islice
dataset = list(islice(streamed_dataset, 25000))

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
# Setup
fasta_path = "input.fasta"
output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from transformers import EsmModel, EsmTokenizer

model_name = "facebook/esm2_t6_8M_UR50D"
tokenizer = EsmTokenizer.from_pretrained(model_name)
model = EsmModel.from_pretrained(model_name)
model.eval().cuda()  # If using a GPU

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 320, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1026, 320, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-5): 6 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=320, out_features=320, bias=True)
            (key): Linear(in_features=320, out_features=320, bias=True)
            (value): Linear(in_features=320, out_features=320, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=320, out_features=320, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
    

In [ ]:
# Output directory for token representations
output_dir = "data/"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("data/token_representations", exist_ok=True)


reps_data = []
master_data = []

for entry in tqdm(dataset):
    # Extract UniRef name (e.g. UniRef50_A0A1H6RCR4)
    name = entry["ids"].split()[0]

    # Flatten multi-line FASTA to single string sequence
    lines = entry["text"].splitlines()
    seq = "".join(line.strip() for line in lines if not line.startswith(">"))

    if len(seq) == 0:
        continue

    # Tokenize and run model
    inputs = tokenizer(seq, return_tensors="pt", truncation=True, max_length=1022)
    inputs = {k: v.cuda() for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Extract representation, removing [CLS]
    last_hidden = outputs.last_hidden_state[0]
    token_reps = last_hidden[1:len(seq)+1].cpu()

    # Save as data/token_representations/UniRef50_XXXX.pt
    pt_filename = f"{name}.pt"
    pt_path = os.path.join("data/token_representations", pt_filename)
    torch.save(token_reps, pt_path)

    # Save relative path
    rel_path = os.path.join("data/token_representations", pt_filename)
    reps_data.append([name, seq, rel_path])

    try:
        analysis = ProteinAnalysis(seq)
        master_data.append([
            name, seq, rel_path, name, "", len(seq),
            token_reps.shape[1], analysis.molecular_weight(), analysis.aromaticity(),
            analysis.instability_index(), analysis.flexibility(),
            analysis.gravy(), analysis.isoelectric_point(), analysis.charge_at_pH(7.0)
        ])
    except Exception as e:
        print(f"Skipping {name} due to ProteinAnalysis error: {e}")
        continue

# Save CSVs
pd.DataFrame(reps_data, columns=["Name", "Sequence", "Token Representations"]).to_csv("data/reps_paths.csv", index=False)
pd.DataFrame(master_data, columns=[
    "Name", "Sequence", "Token Representations", "ID", "Description", "Length",
    "Num_Features", "Molecular Weight", "Aromaticity", "Instability Index",
    "Flexibility", "GRAVY", "Isoelectric Point", "Charge at pH:7.0"
]).to_csv("data/master_tokens_dataframe.csv", index=False)


  0%|          | 8/25000 [00:02<1:21:48,  5.09it/s]

Skipping UniRef50_K7G060 due to ProteinAnalysis error: ''X'' is not a valid unambiguous letter for protein


  0%|          | 18/25000 [00:04<1:17:14,  5.39it/s]

Skipping UniRef50_UPI00244459C2 due to ProteinAnalysis error: ''X'' is not a valid unambiguous letter for protein


  0%|          | 22/25000 [00:05<1:17:42,  5.36it/s]

...

 26%|██▌       | 6517/25000 [28:36<1:57:30,  2.62it/s]

Skipping UniRef50_UPI0018A766D5 due to ProteinAnalysis error: ''X'' is not a valid unambiguous letter for protein


 26%|██▌       | 6525/25000 [28:37<50:04,  6.15it/s]

Skipping UniRef50_UPI0011765B6A due to ProteinAnalysis error: ''X'' is not a valid unambiguous letter for protein


 26%|██▌       | 6556/25000 [28:50<2:26:29,  2.10it/s]

In [ ]:
print(dataset[0].keys())


dict_keys(['ids', 'text'])


In [ ]:
!zip -r dataset25k.zip data/


Streaming output truncated to the last 5000 lines.
  adding: data/token_representations/UniRef50_A0A369SCI2.pt (deflated 7%)
  adding: data/token_representations/UniRef50_H3D3Z4.pt (deflated 7%)
  adding: data/token_representations/UniRef50_UPI0022B6AE3B.pt (deflated 7%)
  adding: data/token_representations/UniRef50_A0A9Q9XUU9.pt (deflated 7%)
  adding: data/token_representations/UniRef50_A0A977KQ20.pt (deflated 7%)
  adding: data/token_representations/UniRef50_B4N0D6.pt (deflated 7%)
  adding: data/token_representations/UniRef50_U2HBY5.pt (deflated 7%)
  adding: data/token_representations/UniRef50_UPI0025FFCD96.pt (deflated 7%)
  adding: data/token_representations/UniRef50_A0A2U1U5Y8.pt (deflated 8%)
  adding: data/token_representations/UniRef50_K8FEQ1.pt (deflated 7%)
  adding: data/token_representations/UniRef50_A0A0G4HRG7.pt (deflated 7%)
  adding: data/token_representations/UniRef50_UPI0023B7D512.pt (deflated 7%)
  adding: data/token_representations/UniRef50_UPI00187CD64B.pt (defl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p "/content/drive/My Drive/ColabBackups"
!cp -r data/ "/content/drive/My Drive/ColabBackups/"

         35.92G 100%    1.21MB/s    7:52:30 (xfr#25002, to-chk=0/25004)
